In [5]:
%matplotlib inline
import json
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas_profiling
import requests
plt.style.use('fivethirtyeight')
DATASET_DIR = '../../datasets/'

d:\programdata\miniconda3\lib\site-packages\pandas_profiling\plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "d:\programdata\miniconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\programdata\miniconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\programdata\miniconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "d:\programdata\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "d:\pro

In [18]:
df = pd.read_csv(
    DATASET_DIR + 'eleicao_rj/votacao_secao_2018_RJ.csv',
    sep=';',
    encoding='Latin1')
df.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS
0,15/10/2018,18:30:07,2018,2,ELEIÇÃO ORDINÁRIA,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,RIO DE JANEIRO,60011,RIO DE JANEIRO,7,402,7,DEPUTADO ESTADUAL,20911,ELITUSALEM GOMES DE FREITAS,1
1,15/10/2018,18:30:07,2018,2,ELEIÇÃO ORDINÁRIA,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,RIO DE JANEIRO,60011,RIO DE JANEIRO,25,968,7,DEPUTADO ESTADUAL,12612,SERGIO BERNARDINO DUARTE,1
2,15/10/2018,18:30:07,2018,2,ELEIÇÃO ORDINÁRIA,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,RIO DE JANEIRO,58467,MESQUITA,150,46,6,DEPUTADO FEDERAL,1000,MARCELO HODGE CRIVELLA,1
3,15/10/2018,18:30:07,2018,2,ELEIÇÃO ORDINÁRIA,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,RIO DE JANEIRO,58637,NILÓPOLIS,221,117,6,DEPUTADO FEDERAL,1000,MARCELO HODGE CRIVELLA,2
4,15/10/2018,18:30:07,2018,2,ELEIÇÃO ORDINÁRIA,1,297,ELEIÇÕES GERAIS ESTADUAIS 2018,07/10/2018,E,...,RIO DE JANEIRO,58432,ITAPERUNA,107,119,6,DEPUTADO FEDERAL,1701,FRANKLIN JORGE SANTOS,1


In [27]:
rj = df.query('DS_CARGO == "GOVERNADOR" and NM_UE == "RIO DE JANEIRO"')
del df
rj2 = rj.drop([
    'DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
    'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO',
    'DT_ELEICAO', 'TP_ABRANGENCIA', 'CD_MUNICIPIO', 'SG_UF', 'SG_UE',
    'NM_UE', 'CD_CARGO', 'DS_CARGO', 'NM_MUNICIPIO', 'NR_VOTAVEL'],
    axis=1)
rj2.columns = ['zona', 'secao','candidato', 'votos'
]
rj2.to_csv(DATASET_DIR + 'eleicao_rj/limpo.csv')
del rj
rj2.loc[:, 'turno'] = 1
rj2.head(5)


,zona,secao,candidato,votos,turno
81,170,423,ANTONIO PEDRO INDIO DA COSTA,8,1
149,153,151,TARCISIO MOTTA DE CARVALHO,8,1
156,238,24,VOTO NULO,54,1
164,38,130,VOTO NULO,33,1
178,254,147,TARCISIO MOTTA DE CARVALHO,10,1


In [11]:
rj = pd.read_csv(
    DATASET_DIR + 'eleicao_rj/resultado_eleicao.csv',
    encoding='Latin1',
    sep=';')
rj.head()

,UF,Zona,Cargo,Número do candidato,Nome do candidato,Nome do candidato (urna),Partido,Coligação,Turno,Situação de totalização,Votos nominais do candidato,Votos válidos por abrangência,% votos válidos,Data última totalização
0,RJ,10,Governador,25,EDUARDO DA COSTA PAES,EDUARDO PAES,DEM,DEM / PP / MDB / PTB / SOLIDARIEDADE / PSDB / ...,2,Não eleito,29584,56420,"52,44",28/10/2018 20:16:57
1,RJ,10,Governador,20,WILSON JOSÉ WITZEL,WILSON WITZEL,PSC,PSC / PROS,2,Eleito,26836,56420,"47,56",28/10/2018 20:16:57
2,RJ,101,Governador,20,WILSON JOSÉ WITZEL,WILSON WITZEL,PSC,PSC / PROS,2,Eleito,6800,9290,"73,2",28/10/2018 20:16:57
3,RJ,101,Governador,25,EDUARDO DA COSTA PAES,EDUARDO PAES,DEM,DEM / PP / MDB / PTB / SOLIDARIEDADE / PSDB / ...,2,Não eleito,2490,9290,"26,8",28/10/2018 20:16:57
4,RJ,102,Governador,20,WILSON JOSÉ WITZEL,WILSON WITZEL,PSC,PSC / PROS,2,Eleito,5085,8706,"58,41",28/10/2018 20:16:57


In [36]:
rj = pd.read_csv(
    DATASET_DIR + 'eleicao_rj/resultado_eleicao.csv',
    encoding='Latin1',
    sep=';')
rj = rj.loc[:, [
    'Nome do candidato', 'Zona', 'Turno', 'Votos nominais do candidato'
]]
rj.columns = ['candidato', 'zona', 'turno', 'votos']
rj.head(5)

,candidato,zona,turno,votos
0,EDUARDO DA COSTA PAES,10,2,29584
1,WILSON JOSÉ WITZEL,10,2,26836
2,WILSON JOSÉ WITZEL,101,2,6800
3,EDUARDO DA COSTA PAES,101,2,2490
4,WILSON JOSÉ WITZEL,102,2,5085


## Limpeza dos dados das zonas eleitorais

In [37]:
end_zonas = pd.read_csv(
    DATASET_DIR + 'eleicao_rj/lista_zonas_eleitorais.csv',
    encoding='Latin1',
    index_col=0)
end_zonas.head()

,sigla_uf,endereco,cep,bairro,nome_municipio,sigla_uf.1
numero_zona,,,,,,
4,19-0004,RUA JARDIM BOTÂNICO 1060,22460000,JARDIM BOTÂNICO,RIO DE JANEIRO,RJ
5,19-0005,"RUA MIGUEL LEMOS, 97",22071000,COPACABANA,RIO DE JANEIRO,RJ
7,19-0007,RUA ANTONIO BASILIO 76,20511190,TIJUCA,RIO DE JANEIRO,RJ
8,19-0008,"AVENIDA DOM HÉLDER CÂMARA, 4175",20771000,DEL CASTILHO,RIO DE JANEIRO,RJ
9,19-0009,AV AYRTON SENNA 2001 BLOCO C,22775002,BARRA DA TIJUCA,RIO DE JANEIRO,RJ


In [38]:
end_zonas = end_zonas.rename(columns={'endereco': 'rua',
                                     'nome_municipio': 'municipio'})
end_zonas = end_zonas.drop(['sigla_uf', 'cep', 'sigla_uf.1'], axis=1)
end_zonas['endereco'] = end_zonas.apply(lambda x: ', '.join(x), axis=1)
colunas = ['rua', 'bairro', 'municipio', 'endereco']
for c in colunas:
    end_zonas.loc[:, c] = end_zonas.loc[:, c].apply(lambda x: x.lower())

erros = []

dados_json = []
url = 'https://maps.googleapis.com/maps/api/geocode/json'
bounds = '-23.371469,-44.917957|-20.784440,-40.504246'
with open('gkey.txt') as f:
    g_key = f.read()
for z, v in end_zonas.iterrows():
    end = v["endereco"].replace(' ', '+')
    response = requests.get(url, {
        'bounds': bounds,
        'key': g_key,
        'address': end
    })
    try:
        coords = json.loads(
            response.text)['results'][0]['geometry']['location']
        end_zonas.loc[z, 'lat'] = coords['lat']
        end_zonas.loc[z, 'lng'] = coords['lng']
        dados_json.append(response.json())
    except:
        erros.append(z)
        print(f'Não achou zona {z}')
    time.sleep(0.05)
end_zonas.loc[60, 'lat'] = -21.954952
end_zonas.loc[60, 'lng'] = -42.136810
end_zonas.loc[196, 'lat'] = -22.151659
end_zonas.loc[196, 'lng'] = -42.926066
end_zonas.index.name = 'zona'

Não achou zona 60


In [39]:
end_zonas.head()

,rua,bairro,municipio,endereco,lat,lng
zona,,,,,,
4,rua jardim botânico 1060,jardim botânico,rio de janeiro,"rua jardim botânico 1060, jardim botânico, rio...",-22.972010,-43.224547
5,"rua miguel lemos, 97",copacabana,rio de janeiro,"rua miguel lemos, 97, copacabana, rio de janeiro",-22.977688,-43.192630
7,rua antonio basilio 76,tijuca,rio de janeiro,"rua antonio basilio 76, tijuca, rio de janeiro",-22.924920,-43.235859
8,"avenida dom hélder câmara, 4175",del castilho,rio de janeiro,"avenida dom hélder câmara, 4175, del castilho,...",-22.881511,-43.274248
9,av ayrton senna 2001 bloco c,barra da tijuca,rio de janeiro,"av ayrton senna 2001 bloco c, barra da tijuca,...",-22.992552,-43.367683


## Limpando os dados do IDH

A partir dos dados do IDH Municipal, agregaremos esses dados na nossa análise de forma para enriquecermos a nossa análise.

In [44]:
idhm = pd.read_excel(
    DATASET_DIR + 'IDHM/IDHM_Municipios_RJ.xlsx', index_col=0)
idhm.head()

,Espacialidades,IDHM 2010
Código,,
76,Brasil,0.727
3300100,Angra dos Reis,0.724
3300159,Aperibé,0.692
3300209,Araruama,0.718
3300225,Areal,0.684


In [45]:
idhm.index.name = 'cod'
idhm.columns = ['municipio', 'idhm']
idhm.loc[:,'municipio'] = idhm.municipio.apply(lambda x: x.lower())
idhm.head()

,municipio,idhm
cod,,
76,brasil,0.727
3300100,angra dos reis,0.724
3300159,aperibé,0.692
3300209,araruama,0.718
3300225,areal,0.684


## Transformação dos dados dos votos
Vamos criar dois datasets diferentes, um contendo o percentual dos votos de cada zona eleitoral foi para cada candidato e o outro contendo a representatividade de votos de cada zona eleitoral em relação ao total de cada candidato.

In [55]:
votos_zona.index.values

array([(4, 1), (4, 2), (5, 1), (5, 2), (7, 1), (7, 2), (8, 1), (8, 2),
       (9, 1), (9, 2), (10, 1), (10, 2), (14, 1), (14, 2), (16, 1),
       (16, 2), (17, 1), (17, 2), (21, 1), (21, 2), (22, 1), (22, 2),
       (23, 1), (23, 2), (24, 1), (24, 2), (25, 1), (25, 2), (26, 1),
       (26, 2), (27, 1), (27, 2), (28, 1), (28, 2), (29, 1), (29, 2),
       (30, 1), (30, 2), (31, 1), (31, 2), (32, 1), (32, 2), (34, 1),
       (34, 2), (35, 1), (35, 2), (36, 1), (36, 2), (37, 1), (37, 2),
       (38, 1), (38, 2), (40, 1), (40, 2), (41, 1), (41, 2), (42, 1),
       (42, 2), (43, 1), (43, 2), (45, 1), (45, 2), (48, 1), (48, 2),
       (49, 1), (49, 2), (50, 1), (50, 2), (51, 1), (51, 2), (52, 1),
       (52, 2), (54, 1), (54, 2), (55, 1), (55, 2), (56, 1), (56, 2),
       (57, 1), (57, 2), (59, 1), (59, 2), (60, 1), (60, 2), (61, 1),
       (61, 2), (62, 1), (62, 2), (63, 1), (63, 2), (64, 1), (64, 2),
       (65, 1), (65, 2), (68, 1), (68, 2), (69, 1), (69, 2), (70, 1),
       (70, 2), (71, 

In [54]:
for z, t in votos_zona.index.values:
    votos.query(f'zona == {z} & turno == {t}')/votos_zona.loc[z].loc[t]
    for i, v in (votos.loc[z] / votos_zona[z]).iterrows():
        votos_perc_zona.loc[z, i] = round(v[0] * 100, 2)
    for i, v in (votos.loc[z] / votos_candidato).iterrows():
        votos_perc_cand.loc[z, i] = round(v[0] * 100, 2)

KeyError: '[1] not in index'

In [40]:
votos = pd.concat([
    rj2.loc[:, ['zona', 'candidato', 'votos', 'turno']].set_index('zona'),
    rj.set_index('zona')
], sort=True)
votos_zona = np.sum(votos.groupby(['zona', 'turno']))['votos']
votos_candidato = np.sum(votos.groupby('candidato')).loc[:, 'votos']
votos = np.sum(votos.groupby(['zona', 'candidato', 'turno']))
votos_perc_zona = pd.DataFrame(columns=list(votos.index.levels[1]))
votos_perc_cand = pd.DataFrame(columns=list(votos.index.levels[1]))
for z, t in votos_zona.index.values:
    votos.query(f'zona == {z} & turno == {t}')/votos_zona.loc[z].loc[t]
    for i, v in (votos.loc[z] / votos_zona[z]).iterrows():
        votos_perc_zona.loc[z, i] = round(v[0] * 100, 2)
    for i, v in (votos.loc[z] / votos_candidato).iterrows():
        votos_perc_cand.loc[z, i] = round(v[0] * 100, 2)
votos_perc_cand.index.name = 'zona'
votos_perc_zona.index.name = 'zona'

d:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: '[1] not in index'

In [ ]:
votos_perc_cand.to_csv(DATASET_DIR + 'eleicao_rj/votos_perc_cand.csv')
votos_perc_zona.to_csv(DATASET_DIR + 'eleicao_rj/votos_perc_zona.csv')

In [ ]:
!git status